In [34]:
## imports
import importlib
import osmosys.osmosys
import osmosys.Backups
import activityinfo
import pandas as pd
import model.modelAI
from IPython.display import display
import json
import os.path
from urllib.error import HTTPError

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo)

<module 'activityinfo' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\activityinfo\\__init__.py'>

In [35]:
month = 'JUNIO'

In [36]:
osmosysData = osmosys.osmosys.get_ai_form_level_1()
print(osmosysData.info())

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   folder              0 non-null      object
 1   form                0 non-null      object
 2   form_id             0 non-null      object
 3   parent_form         0 non-null      object
 4   parent_form_id      0 non-null      object
 5   org_acron           0 non-null      object
 6   id_registro_imp     0 non-null      object
 7   osmosys_code        0 non-null      object
 8   canton              0 non-null      object
 9   canton_id_registro  0 non-null      object
dtypes: object(10)
memory usage: 0.0+ bytes
None


In [31]:
# get all unique form ids
#osmosysData.form_id.value_counts()
parentFormsIds = osmosysData.parent_form_id.unique()
print(parentFormsIds)
print(type(parentFormsIds))
print(osmosysData.groupby(['parent_form', 'parent_form_id']).size())

[]
<class 'numpy.ndarray'>
Series([], dtype: int64)


In [32]:
importlib.reload(osmosys.osmosys)
aiToken = osmosys.osmosys.getToken()
formTotalDf = None
for parentFormsId in parentFormsIds:
    print(parentFormsId)
    client = activityinfo.Client(token=aiToken, base_url='https://www.activityinfo.org/resources')
    dbsJson = client.get_resource('form/{parentFormsId}/query'.format(parentFormsId=parentFormsId))
    dbsDf = pd.DataFrame.from_records(dbsJson)
    dbsDf['parent_form_id'] = parentFormsId
    print(dbsDf.shape)
    if formTotalDf is None:
        formTotalDf = dbsDf
    else:
        formTotalDf = pd.concat([formTotalDf, dbsDf], axis=0)
    print(formTotalDf.shape)

In [33]:
# ordeno para quitar duplicados
# filtro implementaciòn indirecta y doante acnur
print('formTotalDf: ' + str(formTotalDf.shape))
formPartnersDf = formTotalDf.loc[
    (formTotalDf['implementacion'] == 'Indirecta') & (formTotalDf['donante.@id'] == 'chgxs46l02wy5i81e')]
print('formPartnersDf: ' + str(formPartnersDf.shape))

AttributeError: 'NoneType' object has no attribute 'shape'

In [24]:
formPartnersDfClean = formPartnersDf.sort_values('@lastEditTime', ascending=False).drop_duplicates(
    subset=['implementador.@id', 'ubicacion.@id', 'parent_form_id'])
print('formPartnersDfClean: ' + str(formPartnersDfClean.shape))
formPartnersDfClean = formPartnersDfClean[
    ["@id", "@lastEditTime", "implementacion", "donante.@id", "donante.org_nombre", "donante.donante",
     "implementador.@id",
     "implementador.org_nombre", "implementador.donante", "ubicacion.@id", "ubicacion.name", "ubicacion.code",
     "ubicacion.parent.name", "ubicacion.parent.code", "codigo_referencia", "parent_form_id"
     ]]

formPartnersDfClean: (139, 40)


In [25]:
## hago merge para buscar cuales falta por crear
mergeDf = pd.merge(osmosysData, formPartnersDfClean, how='left',
                   left_on=['parent_form_id', 'id_registro_imp', 'canton_id_registro'],
                   right_on=['parent_form_id', 'implementador.@id', 'ubicacion.@id'])
formsToCreate = mergeDf[mergeDf['@id'].isna()]
numberOfFormTocreate = formsToCreate.shape[0]
print('Forms to create: ' + str(numberOfFormTocreate))

Forms to create: 142


In [26]:
importlib.reload(model.modelAI)

## creo las estructuras de datos
changesList = []
newIds = []
for index, row in formsToCreate.iterrows():
    form = model.modelAI.Form(cqes12jkykah3la9='40038', org_user='cc9fjcdkyyk18oj6', implementacion='c89eykqkykc8oig1b',
                              donante='chgxs46l02wy5i81e',
                              implementador=row['id_registro_imp'],
                              ubicacion=row['canton_id_registro'])
    newId = activityinfo.generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(row['parent_form_id'], newId, None, form)
    changesList.append(record)

changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

In [27]:
osmosys.Backups.do_backup('form1_canton_org','',month,2022, changesList, finalJson)

<class 'list'>
 creado respaldo: 2022-JUNIO\data_form1_canton_org-.json


In [109]:
## send to AI
try:
    if (numberOfFormTocreate > 0):
        client.post_resource('update', body=finalJson)
        print("Enviado a AI")
    else:
        print("nada por crear en AI")
except HTTPError as e:
    code = e.response.status_code
    print('error:')
    print(code)
    print(e)

nada por crear en AI
